In [ ]:
!curl -L -O https://github.com/YutaroOgawa/pytorch_advanced/raw/master/1_image_classification/data/goldenretriever-3724972_640.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 87927  100 87927    0     0   297k      0 --:--:-- --:--:-- --:--:--  297k


In [ ]:
from  PIL import Image
import torchvision
import torch
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from torchvision import transforms
from torch.utils import data

In [ ]:
dir = os.getcwd()
file_name = os.path.join(dir+'/*.jpg')
for path in glob.glob(file_name):
    path

In [ ]:
img = Image.open(path).convert('RGB')

In [ ]:
dataset_url = 'https://s3.amazonaws.com/fast-ai-imageclas/imagenette-160.tgz'
# !curl -L -O https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

torchvision.datasets.utils.download_url('https://s3.amazonaws.com/fast-ai-imageclas/imagenette2-160.tgz',os.getcwd())

Using downloaded and verified file: /content/imagenette2-160.tgz


In [ ]:
import tarfile
with tarfile.open('/content/imagenette2-160.tgz', 'r:gz') as tar:
    tar.extractall(path = './data')

In [ ]:
def make_file_list(phase):
    dir = '/content/data/imagenette2-160/'
    file_list = []
    for path in glob.glob(dir+phase+'/**/*.JPEG'):
        file_list.append(path)
    return file_list    

In [ ]:
train_file_list = make_file_list(phase = 'train')
val_file_list = make_file_list(phase = 'val')

In [ ]:
'/'.join(train_file_list[0].split('/')[-3:])
val_file_list[0][34:43]

'n01440764'

In [ ]:

class ImageTransform():
    def __init__(self,resize,mean,std):
        self.data_transform = {
            'train': transforms.Compose([
            transforms.RandomResizedCrop(resize, scale=(0.5,1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val' : transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])
        }
    def __call__(self, img, phase = 'train'):
        return self.data_transform[phase](img)

In [ ]:
import pandas as pd
csv = pd.read_csv('/content/data/imagenette2-160/noisy_imagenette.csv')

In [ ]:
# csv.loc[:,:]
label = csv.loc[csv['path'] == train_file_list[0][30:]]['noisy_labels_0']
label

2266    n01440764
Name: noisy_labels_0, dtype: object

In [ ]:
class Dataset(data.Dataset):
    def __init__(self, path, transform = None, phase = 'train'):
        self.path = path
        self.transform = transform
        self.phase = phase
        self.noisy_label = pd.read_csv('/content/data/imagenette2-160/noisy_imagenette.csv')

    def __len__(self):
        return len(self.path)

    def __getitem__(self, index):
        
        img = Image.open(self.path[index]).convert('RGB')
        tf_img = self.transform(img, self.phase)
        if self.phase == 'train':
            if self.path[index][36:45] == 'n01440764':
                label = 1
            else :
                label = 0
        elif self.phase == 'val':
            if self.path[index][34:43] == 'n01440764':
                label = 1
            else :
                label = 0

        return tf_img, label

In [ ]:
resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
train_dataset = Dataset(train_file_list, transform=ImageTransform(resize, mean, std), phase='train')
val_dataset = Dataset(val_file_list, transform= ImageTransform(resize, mean, std), phase='val')

In [ ]:
train_dataset.__getitem__(200)

(tensor([[[-1.0562, -1.1932, -1.2617,  ..., -0.4739, -0.3198, -0.3883],
          [-0.9363, -1.2445, -1.3302,  ..., -0.4739, -0.4911, -0.6794],
          [-0.5082, -1.1247, -1.3302,  ..., -0.5253, -0.7137, -1.0562],
          ...,
          [-0.8849, -1.1418, -1.2959,  ..., -0.9534, -1.0048, -1.3987],
          [-1.2959, -1.3302, -1.3302,  ..., -0.9020, -0.9020, -1.1932],
          [-1.7069, -1.5185, -1.3473,  ..., -0.9020, -0.8678, -1.0048]],
 
         [[-0.7402, -0.8803, -0.9678,  ..., -0.4951, -0.3725, -0.4426],
          [-0.6176, -0.9153, -1.0553,  ..., -0.4951, -0.5301, -0.7227],
          [-0.1800, -0.7927, -1.0553,  ..., -0.5476, -0.7402, -1.0728],
          ...,
          [-0.6176, -0.8803, -0.9853,  ..., -0.8102, -0.9328, -1.3354],
          [-1.0378, -1.0728, -1.0378,  ..., -0.7227, -0.7927, -1.0903],
          [-1.4580, -1.2654, -1.0553,  ..., -0.7052, -0.7227, -0.8627]],
 
         [[-0.8981, -1.0550, -1.1596,  ..., -0.7413, -0.5670, -0.6541],
          [-0.7936, -1.1073,

In [ ]:
batch_size = 64
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle= True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle= False)
dataloader_dict = {}
dataloader_dict['train'] = train_dataloader
dataloader_dict['val'] = val_dataloader

In [ ]:
from torchvision.models.vgg import VGG16_Weights
model = torchvision.models.vgg16(weights = VGG16_Weights)

In [ ]:
import torch.nn as nn
from tqdm import tqdm

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
for name, param in model.named_parameters():
    param.requires_grad = False
model.classifier[6] = nn.Linear(in_features = 4096, out_features = 2)

In [ ]:
next(model.classifier[6].named_parameters())

('weight', Parameter containing:
 tensor([[ 0.0150, -0.0063,  0.0080,  ...,  0.0149,  0.0038, -0.0065],
         [-0.0038,  0.0042, -0.0130,  ..., -0.0018, -0.0113, -0.0033]],
        requires_grad=True))

In [ ]:
params_to_update = []
for name, param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
params_to_update
optimizer = torch.optim.AdamW(params = params_to_update, lr= 1e-3)

In [ ]:
def train_model(model, dataloader_dict, criterion, optimizer, num_epochs):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch +1} / {num_epochs} ')
        print('----------------------------------')

        for phase in ['train', 'val']:

            if phase == 'train':
                model.train()
            elif phase == 'val':
                model.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            if epoch == 0 and phase == 'train':
                continue

            for inputs, labels in tqdm(dataloader_dict[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()

                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, pred = torch.max(outputs,1)

                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                        epoch_loss += loss.item() * inputs.size(0)
                        epoch_corrects += torch.sum(pred == labels.data)
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double()/ len(dataloader_dict[phase].dataset)
            print(f'{phase}, {epoch_loss}, {epoch_acc}')

            

In [ ]:
num_epochs = 2
train_model(model, dataloader_dict, criterion, optimizer,num_epochs = num_epochs)

Epoch 1 / 2 
----------------------------------


100%|██████████| 62/62 [00:31<00:00,  1.99it/s]


val, 0.6320455335811445, 0.6601273885350318
Epoch 2 / 2 
----------------------------------


100%|██████████| 148/148 [01:13<00:00,  2.01it/s]


train, 0.021534697699564342, 0.9907065159995776


100%|██████████| 62/62 [00:31<00:00,  1.98it/s]

val, 0.005804692734341333, 0.9984713375796177
